In [ ]:
import numpy as np
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import axes3d

## 学習
1. ニューラルネットワークの出力の精度を高めたい。（＝損失関数の結果を 0 に近づける）
1. 重みのパラメータを変化させると損失関数の結果も変化する。
1. 損失関数を重みパラメータで微分すると、損失関数が 0 に近づくような重みパラメータを探ることが出来る

## 微分
$$
\frac{df(x)}{dx} = \lim_{h \to 0}\frac{f(x + h) - f(x)}{h}
$$
機械学習的には微分というよりも、ある一点（重み $x$）での傾きがわかれば良いので解析的に微分するよりは、十分に小さな $h$（大体 $10^{-4}$）程度の数値微分で代用する。
$x+h$だけではなく $x-h$との中心を求めて（中心差分）、誤差を小さくする工夫が採られる。

In [ ]:
def numerical_diff(f, x):
    h = 1e-4
    return (f(x+h) - f(x-h)) / (h*2)

In [ ]:
def func_1(x):
    return 0.01 * x ** 2 + 0.1 * x

def func_1_prime(x):
    return 0.01 * 2 * x + 0.1

x0 = np.arange(0.0, 10.0, 0.1)
y0 = func_1(x0)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.plot(x0, y0)
plt.show()

In [ ]:
print("numerical_diff: %f, prime %f" % (numerical_diff(func_1, 5), func_1_prime(5)))

In [ ]:
print("numerical_diff: %f, prime %f" % (numerical_diff(func_1, 10), func_1_prime(10)))

## 偏微分
例えば以下のような変数が複数ある($x_0, x_1$)関数があるとして、
$$
f(x_0, x_1) = x_0^2 + x_1^2
$$


In [ ]:
def func_2(x) -> float:
    return x[0] ** 2 + x[1] ** 2

x0, x1 = np.meshgrid(np.arange(-3, 3, 0.1), np.arange(-3, 3, 0.1))
y1 = func_2([x0, x1])

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_xlabel("x0")
ax.set_ylabel("x1")
ax.set_zlabel("f(x0, x1)")
ax.plot_wireframe(x0, x1, y1)
plt.show()

$x_0, x_1$ それぞれの変数毎に微分する
$$
\frac{\partial f}{\partial x_0} = 2x_0 \\
\frac{\partial f}{\partial x_1} = 2x_1
$$

In [ ]:
def func_2_0(x):
    return x ** 2.0
def func_2_prime(x):
    return 2 * x

print("numerical_diff: %f, prime %f" % (numerical_diff(func_2_0, 2), func_2_prime(2)))
print("numerical_diff: %f, prime %f" % (numerical_diff(func_2_0, 3), func_2_prime(3)))

## 勾配
学習にあたっては偏微分をまとめて計算したいので、偏微分をまとめてベクトルとする。これを勾配(gradient)という。
上のような関数であれば、
$$
\left(
  \begin{array}{cc}
    \frac{\partial f}{\partial x_0} & \frac{\partial f}{\partial x_1}
\end{array}
\right)
$$
として扱う。

In [ ]:
def numerical_gradient(f, x):
    return np.array([numerical_diff(func_2_0, i) for i in x])

In [ ]:
    x0 = np.arange(-2, 2.5, 0.25)
    x1 = np.arange(-2, 2.5, 0.25)
    X, Y = np.meshgrid(x0, x1)
    
    X = X.flatten()
    Y = Y.flatten()
    
    grad = numerical_gradient(func_2, np.array([X, Y]))
    
    plt.figure()
    plt.quiver(X, Y, -grad[0], -grad[1],  angles="xy",color="#666666")
    plt.xlim([-2, 2])
    plt.ylim([-2, 2])
    plt.xlabel('x0')
    plt.ylabel('x1')
    plt.grid()
    # plt.legend()
    plt.draw()
    plt.show()


上の関数の場合勾配の結果にマイナスを掛けると、勾配のベクトルの向きは、関数の値を最大限減らす方向を向いている。
ただし、実際のところ勾配が向いている方向は、傾きゼロの極値の方向なので、必ずしも最小値が求まるとは限らない。

この性質を利用して最適パラメータを探索（学習）する手法を勾配法(gradient method)という。
勾配法では、勾配の方向に一定の距離だけ進む事を繰り返して、最小値を探索する。
$$
x_0 = x_0 - \eta\frac{\partial f}{\partial x_0} \\
x_1 = x_1 - \eta\frac{\partial f}{\partial x_1}
$$

$\eta$ は更新量を表し学習率と呼ばれる。一回の学習でどの程度値を更新するかを定めるパラメータ。重みパラメータとは意味合いが違い、大抵は人が適当に決める。他と区別するためにハイパーパラメータと呼ばれる。

In [ ]:
# 勾配降下法
def gradient_descent(f, init_x, lr = 0.01, step_num = 100):
    xl = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, xl)
        xl -= lr * grad
        
    return xl

In [ ]:
gradient_descent(func_2, np.array([-3.0, 4.0]), 0.1, 100)

学習率は大きすぎると発散してしまい、小さすぎると更新しきれない。だから、勾配法を使う場合には適切な学習率が設定されているかどうかをきちんと見極める必要がある。

In [ ]:
# 大きすぎる lr = 10.0
print("lr = 10.0: %s" % gradient_descent(func_2, np.array([-3.0, 4.0]), 10.0, 100))

# 小さすぎる lr = 1e-10
print("lr = 1e-10: %s" % gradient_descent(func_2, np.array([-3.0, 4.0]), 1e-10, 100))